In [1]:
import json
import logging

import numpy as np

from visualized_ml.gaussian_mixture.univariate import (
    UnivariateGaussianMixture,
    learn_em,
)

# setup

## logging

In [2]:
class UnivariateGaussianMixtureHandler(logging.Handler):
    def emit(self, record):
#         print(record.data["improvement"])
        if record.data["iteration"] == 1:
            self._previous_value = -1e100
            self._iterations = ()
            self._likelihoods = ()
            self._improvements = ()
        self._iterations = self._iterations + (record.data["iteration"],)
        self._likelihoods = self._likelihoods + (record.data["average_log_likelihood"],)
        self._improvements = self._improvements + (record.data["improvement"],)
        improvement_since_last = record.data["average_log_likelihood"] - self._previous_value
        ignore = improvement_since_last < 0.01
        ignore = ignore and not record.data["is_done"]
        ignore = ignore and not len(self._iterations) > 100
        if ignore:
            return
        update_learned_gmm_plots(record.data["gaussian_mixture"])
        update_progress_plot(
            self._iterations,
            self._likelihoods,
            self._improvements)
        # reset history
        self._previous_value = record.data["average_log_likelihood"]
        self._iterations = ()
        self._likelihoods = ()
        self._improvements = ()
        return


logging.basicConfig(level='DEBUG')
logging.getLogger("visualized_ml.gaussian_mixture.univariate").addHandler(UnivariateGaussianMixtureHandler())
logger = logging.getLogger(__name__)


In [3]:
logging.getLogger().handlers

[<StreamHandler stderr (NOTSET)>]

In [4]:
logging.getLogger("visualized_ml.gaussian_mixture.univariate").handlers

[<UnivariateGaussianMixtureHandler (NOTSET)>]

In [5]:
logging.getLogger(__name__).handlers

[]

## data

In [6]:
np.set_printoptions(suppress=True)
np_rng = np.random.default_rng(seed=0)


In [7]:
k = 10
alpha_dirichlet = np.ones(shape=(k,)) * 5
weights = np_rng.dirichlet(alpha=alpha_dirichlet)
locs = np_rng.normal(size=(k,)) * 5
scales = np.abs(np_rng.normal(size=(k,)))
oracle = UnivariateGaussianMixture(weights, locs, scales)

logger.info("oracle: %s", oracle)

INFO:__main__:oracle: GaussianMixture(
	weights=array([0.1164088 , 0.14579078, 0.0848298 , 0.19046259, 0.07783691,
       0.0811347 , 0.0289743 , 0.05791418, 0.08446264, 0.13218531]),
	locs=array([-0.64267331,  6.83231735, -3.32597337,  1.75755035,  4.51735091,
        0.47006149, -3.71749625, -4.60862688, -2.28862913,  1.10097562]),
	scales=array([1.00961818, 0.20917557, 0.15922501, 0.54084558, 0.21465912,
       0.35537271, 0.65382861, 0.12961363, 0.78397547, 1.49343115])
)


In [8]:
x = oracle.sample(seed=1, n=5_000)


# figure

In [9]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [10]:
x_axis_min_and_max = (min(x), max(x))

In [11]:
data_hist = go.Histogram(
    name='data',
    x=x,
    nbinsx=100,
#     histnorm='probability'
)


def make_empty_gmm_pointwise_plot(name):
    return go.Scatter(
        name=name,
        x=[],
        y=[],
        mode='markers',
        error_x=dict(
            type='data',
            array=[],
            thickness=1,
        ),
        marker=dict(
            size=10,
            color=px.colors.qualitative.Plotly,
            symbol='circle',
            line=dict(
                color='black',
                width=3
            )
    ))


def make_empty_gmm_density_plot(name):
    x_values = np.linspace(*x_axis_min_and_max, num=201)
    y_values = np.zeros_like(x_values)
    return go.Scatter(
        name=name,
        x=x_values,
        y=y_values,
        mode='lines',
#         line_shape='spline',
    )


In [12]:
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=[
        "data",
        "generative gaussian mixture (complete density)",
        "learned gaussian mixture (complete density)",
        "learned gaussian mixture (individual components)",
    ],
    vertical_spacing=0.05,
    specs=[[dict()], [dict(secondary_y=True)]]
)
fig.update_layout(height=700, title_text="gaussian mixture learning")
# fig.update_layout(legend=dict(x=1, y=1))
fig.update_layout(
    legend=dict(x=0, y=1),
#     showlegend=False
)
fig.add_trace(data_hist, row=1, col=1)
fig.add_trace(make_empty_gmm_density_plot('generative gmm'), row=2, col=1)
fig.add_trace(make_empty_gmm_density_plot('learned gmm'), row=2, col=1)
fig.add_trace(make_empty_gmm_pointwise_plot('learned gmm (point-wise)'), row=2, col=1, secondary_y=True)
fig.update_xaxes(range=x_axis_min_and_max)
fig.update_yaxes(
    title_text="density",
#     range=[-2, 0],
#     type="log",
    row=2,
    col=1,
    secondary_y=False
)
fig.update_yaxes(
    title_text="mixture weight",
    range=[0, 1],
#     type="log",
    row=2,
    col=1,
    secondary_y=True
)
# fig.update_yaxes(
#     title_text="mixture weights",
#     range=[-3, 0],
#     type="log",
#     row=4,
#     col=1)

fig = go.FigureWidget(fig)


def update_gmm_pointwise_plot(gmm, trace):
    fig.data[trace].x = gmm.locs
    fig.data[trace].y = gmm.weights
    fig.data[trace].error_x.array = gmm.scales
    return


def update_gmm_density_plot(gmm, trace):
    x_values = fig.data[trace].x
    y_values = gmm.pdf(x_values)
    fig.data[trace].y = y_values
    return


def set_oracle_plots(gmm):
    update_gmm_density_plot(gmm, trace=1)
#     update_gmm_pointwise_plot(gmm, trace=1)
    return


def update_learned_gmm_plots(gmm):
    update_gmm_density_plot(gmm, trace=2)
    update_gmm_pointwise_plot(gmm, trace=3)
    return



In [13]:
def make_empty_line_plot():
    return go.Scatter(x=[], y=[], mode='lines')


fig_progress = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=[
        "average_log_likelihood",
        "average_log_likelihood improvement per iteration"],
    vertical_spacing=0.05,
)
fig_progress.update_layout(height=600, showlegend=False)
fig_progress.add_trace(make_empty_line_plot(), row=1, col=1)
fig_progress.add_trace(make_empty_line_plot(), row=2, col=1)
# fig_progress.update_xaxes(title_text='iteration #')
fig_progress.update_yaxes(
    type="log",
    row=2,
    col=1)


fig_progress = go.FigureWidget(fig_progress)


def update_progress_plot(iterations, log_likelihoods, improvements):
    fig_progress.data[0].x = fig_progress.data[0].x + iterations
    fig_progress.data[0].y = fig_progress.data[0].y + log_likelihoods
    fig_progress.data[1].x = fig_progress.data[1].x + iterations
    fig_progress.data[1].y = fig_progress.data[1].y + improvements
    return


In [14]:
fig

FigureWidget({
    'data': [{'name': 'data',
              'nbinsx': 100,
              'type': 'histogram',
 …

In [15]:
fig_progress

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': '6a8437c0-…

In [16]:
set_oracle_plots(oracle)

# train the thing

In [17]:
gmm_learned = learn_em(x, k, oracle, max_iter=1e10)


DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=1, improvement=3.050492e+00
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=2, improvement=2.570755e-02
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=3, improvement=1.901146e-02
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=4, improvement=1.267618e-02
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=5, improvement=9.773586e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6, improvement=9.088866e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=7, improvement=8.721550e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=8, improvement=7.233928e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=9, improvement=4.499482e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=10, improvement=2.086770e-03
DEBUG:visualized_ml.gaussian_mixture.univariate.l

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=84, improvement=6.242730e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=85, improvement=5.725694e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=86, improvement=5.332960e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=87, improvement=5.030911e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=88, improvement=4.793824e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=89, improvement=4.602373e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=90, improvement=4.442276e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=91, improvement=4.303148e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=92, improvement=4.177569e-05
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=93, improvement=4.060341e-05
DEBUG:visualized_ml.gaussian_mixture.uni

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=167, improvement=1.602213e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=168, improvement=1.486568e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=169, improvement=1.380503e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=170, improvement=1.283336e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=171, improvement=1.194413e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=172, improvement=1.113113e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=173, improvement=1.038846e-06
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=174, improvement=9.710583e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=175, improvement=9.092276e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=176, improvement=8.528664e-07
DEBUG:visualized_ml.gaussian_m

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=249, improvement=2.632497e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=250, improvement=2.627135e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=251, improvement=2.621859e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=252, improvement=2.616667e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=253, improvement=2.611555e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=254, improvement=2.606521e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=255, improvement=2.601562e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=256, improvement=2.596675e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=257, improvement=2.591859e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=258, improvement=2.587111e-07
DEBUG:visualized_ml.gaussian_m

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=331, improvement=2.368277e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=332, improvement=2.366630e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=333, improvement=2.365013e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=334, improvement=2.363425e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=335, improvement=2.361867e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=336, improvement=2.360338e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=337, improvement=2.358839e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=338, improvement=2.357368e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=339, improvement=2.355927e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=340, improvement=2.354514e-07
DEBUG:visualized_ml.gaussian_m

...

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6235, improvement=3.977920e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6236, improvement=4.003126e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6237, improvement=4.028377e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6238, improvement=4.053669e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6239, improvement=4.078994e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6240, improvement=4.104347e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6241, improvement=4.129722e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6242, improvement=4.155111e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6243, improvement=4.180509e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6244, improvement=4.205907e-07
DEBUG:visualized_ml.

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6317, improvement=5.318123e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6318, improvement=5.311060e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6319, improvement=5.303113e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6320, improvement=5.294278e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6321, improvement=5.284552e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6322, improvement=5.273933e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6323, improvement=5.262419e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6324, improvement=5.250010e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6325, improvement=5.236705e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6326, improvement=5.222504e-07
DEBUG:visualized_ml.

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6399, improvement=2.627607e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6400, improvement=2.587808e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6401, improvement=2.548317e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6402, improvement=2.509145e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6403, improvement=2.470300e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6404, improvement=2.431791e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6405, improvement=2.393626e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6406, improvement=2.355814e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6407, improvement=2.318361e-07
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6408, improvement=2.281274e-07
DEBUG:visualized_ml.

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6481, improvement=6.096287e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6482, improvement=5.984171e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6483, improvement=5.874225e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6484, improvement=5.766411e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6485, improvement=5.660691e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6486, improvement=5.557028e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6487, improvement=5.455385e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6488, improvement=5.355727e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6489, improvement=5.258016e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6490, improvement=5.162219e-08
DEBUG:visualized_ml.

DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6563, improvement=1.534760e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6564, improvement=1.513391e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6565, improvement=1.492456e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6566, improvement=1.471947e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6567, improvement=1.451854e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6568, improvement=1.432171e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6569, improvement=1.412888e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6570, improvement=1.393997e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6571, improvement=1.375491e-08
DEBUG:visualized_ml.gaussian_mixture.univariate.learn_em:iteration=6572, improvement=1.357362e-08
DEBUG:visualized_ml.

In [18]:
print("done. learned gaussian mixture:")
print(gmm_learned)

print("and the 'oracle' (truth) was:")
print(oracle)

done. learned gaussian mixture:
GaussianMixture(
	weights=array([0.10922386, 0.00851708, 0.07289014, 0.21807746, 0.03444135,
       0.02387449, 0.14359971, 0.06311343, 0.09225972, 0.23400277]),
	locs=array([ 2.09171232, -0.65848524,  1.58686405,  0.49820001,  2.47849448,
        4.45777278,  6.84419596,  4.51867511, -1.35127113, -3.49420914]),
	scales=array([0.44210852, 0.0658009 , 0.2891626 , 0.5281048 , 0.87570175,
       0.37659445, 0.2096797 , 0.17170632, 0.52085518, 0.838037  ])
)
and the 'oracle' (truth) was:
GaussianMixture(
	weights=array([0.1164088 , 0.14579078, 0.0848298 , 0.19046259, 0.07783691,
       0.0811347 , 0.0289743 , 0.05791418, 0.08446264, 0.13218531]),
	locs=array([-0.64267331,  6.83231735, -3.32597337,  1.75755035,  4.51735091,
        0.47006149, -3.71749625, -4.60862688, -2.28862913,  1.10097562]),
	scales=array([1.00961818, 0.20917557, 0.15922501, 0.54084558, 0.21465912,
       0.35537271, 0.65382861, 0.12961363, 0.78397547, 1.49343115])
)
